# Segragate your tenants by namespace and by nodes

### Prerequisites
1. An AKS cluster (you can create them on the console or via the CLI)
2. az cli installed and logged in
3. A storage account with two folders. Example: \
    ngjasonmultistorage (storage account) \
        - tenanta (container) \
        - tenantb (container)
4. Upload random files to tenanta and tenantb container
5. You have a bash kernel if you are running on windows wsl

### What you will create
1. Create nodes with taints for each specific tenant
2. Create tolerations and node selectors on pod manifest so that they get deployed on the respective tenant node pool (app isolation)
3. Create managed identities, the pods will use the identity via WorkloadIdentityCredential to connect to blob storage
4. Create service accounts to manage federated credentials
5. Create RBAC and ABAC and assigned them to respective managed identities

![image.png](image.png)

## Connect to your cluster and export variables. Replace with your own values

In [ ]:
%env cluster_name=ngjasonaks-cluster
%env resource_group=aks-gpu
%env storage_account_name=ngjasonmultistorage
%env tenant_a_container=tenanta
%env tenant_b_container=tenantb
%env subscription_id=bb608350-d9c5-4b2a-b61e-6df4034dbf07

In [ ]:
%%bash
echo $cluster_name
echo $resource_group
echo $storage_account_name
echo $tenant_a_container
echo $tenant_b_container
echo $subscription_id

## Connect to your cluster - Open a terminal where this jupyter notebook is running and run the command so that kubectl persists across all cells.

```
az aks get-credentials --name $cluster_name -g $resource_group
```

## Enable workload identity on the cluster (This is a one-time setup and can take a while to complete, skip if you have already enabled this before)

In [ ]:
%%bash
az aks update --resource-group $resource_group --name $cluster_name \
  --enable-oidc-issuer \
  --enable-workload-identity


## Create namespaces for each tenant

In [ ]:
%%bash
kubectl create namespace tenant-a
kubectl create namespace tenant-b

## Create two node pools, one for each tenant

In [ ]:
%%bash
az aks nodepool add \
  --name tenantpoola \
  --cluster-name $cluster_name \
  --resource-group $resource_group \
  --node-taints tenantid=tenanta:NoSchedule \
  --labels tenantid=tenanta

az aks nodepool add \
  --name tenantpoolb \
  --cluster-name $cluster_name \
  --resource-group $resource_group \
  --node-taints tenantid=tenantb:NoSchedule \
  --labels tenantid=tenantb

## Create managed identities for each tenant

In [ ]:
%%bash
az identity create --resource-group aks-gpu --name mi-tenant-a
az identity create --resource-group aks-gpu --name mi-tenant-b

## Create RBAC and ABAC policies for each container within blob storage allowing managed identity to connect

In [ ]:
%%bash
az role assignment create \
  --assignee $(az identity show --name mi-tenant-a --resource-group aks-gpu --query principalId -o tsv) \
  --role "Storage Blob Data Contributor" \
  --scope "/subscriptions/${subscription_id}/resourceGroups/${resource_group}/providers/Microsoft.Storage/storageAccounts/${storage_account_name}/blobServices/default/containers/tenanta" \
  --condition "((!(ActionMatches{'Microsoft.Storage/storageAccounts/blobServices/containers/blobs/*'})) OR (@Resource[Microsoft.Storage/storageAccounts/blobServices/containers:name] StringEquals 'tenanta'))" \
  --condition-version "2.0"
  
az role assignment create \
  --assignee $(az identity show --name mi-tenant-b --resource-group aks-gpu --query principalId -o tsv) \
  --role "Storage Blob Data Contributor" \
  --scope "/subscriptions/${subscription_id}/resourceGroups/${resource_group}/providers/Microsoft.Storage/storageAccounts/${storage_account_name}/blobServices/default/containers/tenantb" \
  --condition "((!(ActionMatches{'Microsoft.Storage/storageAccounts/blobServices/containers/blobs/*'})) OR (@Resource[Microsoft.Storage/storageAccounts/blobServices/containers:name] StringEquals 'tenantb'))" \
  --condition-version "2.0"

## Create Service account and federated credential. Pods will use service account to obtain temporary credentials via OIDC

In [ ]:
%%bash
export tenant_a_identity_id=$(az identity show --name mi-tenant-a --resource-group $resource_group --query clientId -o tsv)
export tenant_b_identity_id=$(az identity show --name mi-tenant-b --resource-group $resource_group --query clientId -o tsv)

kubectl apply -n tenant-a -f - <<EOF
apiVersion: v1
kind: ServiceAccount
metadata:
  name: storage-access-sa-tenant-a
  namespace: tenant-a
  annotations:
    azure.workload.identity/client-id: $tenant_a_identity_id
  labels:
    azure.workload.identity/use: "true"
EOF

kubectl apply -n tenant-b -f - <<EOF
apiVersion: v1
kind: ServiceAccount
metadata:
  name: storage-access-sa-tenant-b
  namespace: tenant-b
  annotations:
    azure.workload.identity/client-id: $tenant_b_identity_id
  labels:
    azure.workload.identity/use: "true"
EOF

In [ ]:
%%bash
export SERVICE_ACCOUNT_ISSUER=$(az aks show \
  --resource-group $resource_group \
  --name $cluster_name \
  --query oidcIssuerProfile.issuerUrl -o tsv)

echo $SERVICE_ACCOUNT_ISSUER

az identity federated-credential create \
  --name "aks-federated-credential" \
  --identity-name mi-tenant-a \
  --resource-group $resource_group \
  --issuer $SERVICE_ACCOUNT_ISSUER \
  --subject "system:serviceaccount:tenant-a:storage-access-sa-tenant-a"

az identity federated-credential create \
  --name "aks-federated-credential" \
  --identity-name mi-tenant-b \
  --resource-group $resource_group \
  --issuer $SERVICE_ACCOUNT_ISSUER \
  --subject "system:serviceaccount:tenant-b:storage-access-sa-tenant-b"

## Create test pods

## Check that pods are scheduled on the correct node

In [ ]:
%%bash

kubectl apply -n tenant-a -f - <<EOF
apiVersion: apps/v1
kind: Deployment
metadata:
  name: tenant-a-app
  labels:
    azure.workload.identity/use: "true"  # Required for workload identity
spec:
  selector:
    matchLabels:
      tenantid: tenanta
  template:
    metadata:
      labels:
        tenantid: tenanta
        azure.workload.identity/use: "true"  # Must be on pod template
    spec:
      serviceAccountName: storage-access-sa-tenant-a  # Reference your service account
      nodeSelector:
        tenantid: tenanta
      tolerations:
      - key: "tenantid"
        operator: "Equal"
        value: "tenanta"
        effect: "NoSchedule"
      containers:
      - name: nginx
        image: nginx
EOF

kubectl apply -n tenant-b -f - <<EOF
apiVersion: apps/v1
kind: Deployment
metadata:
  name: tenant-b-app
  labels:
    azure.workload.identity/use: "true"  # Required for workload identity
spec:
  selector:
    matchLabels:
      tenantid: tenantb
  template:
    metadata:
      labels:
        tenantid: tenantb
        azure.workload.identity/use: "true"  # Must be on pod template
    spec:
      serviceAccountName: storage-access-sa-tenant-b  # Reference your service account
      nodeSelector:
        tenantid: tenantb
      tolerations:
      - key: "tenantid"
        operator: "Equal"
        value: "tenantb"
        effect: "NoSchedule"
      containers:
      - name: nginx
        image: nginx
EOF

### If you see your pods stuck in pending state, it is most likely that you dont have the correct labels on the nodes. Check from the portal if the taints and labels are set correctly

In [ ]:
%%bash
kubectl get pods -o wide -n tenant-a
kubectl get pods -o wide -n tenant-b

## Test out the tenant isolation. Open your terminal and run the following commands to execute a bash session into tenant-a pod


In [ ]:
%%bash
export tenant_a_pod=$(kubectl get pods -n tenant-a -o jsonpath='{.items[0].metadata.name}')
kubectl cp -n tenant-a  ./app.py $tenant_a_pod:/tmp/app.py
kubectl cp -n tenant-a  ./install.sh $tenant_a_pod:/tmp/install.sh
kubectl exec -it pod/$tenant_a_pod -n tenant-a -- /bin/bash -c "chmod +x /tmp/install.sh && /tmp/install.sh"

In [ ]:
%%bash
export tenant_b_pod=$(kubectl get pods -n tenant-b -o jsonpath='{.items[0].metadata.name}')
kubectl cp -n tenant-b  ./app.py $tenant_b_pod:/tmp/app.py
kubectl exec -it pod/$tenant_b_pod -n tenant-b -- /bin/bash -c "chmod +x /tmp/install.sh && /tmp/install.sh"

### Test connection to blob storage.

Expected results:\
pod A -> tenant A container ✅ \
pod A -> tenant B container ❌ \
\
pod B -> tenant A container ❌ \
pod B -> tenant B container ✅

In [ ]:
%%bash
export tenant_a_pod=$(kubectl get pods -n tenant-a -o jsonpath='{.items[0].metadata.name}')
kubectl exec -it pod/$tenant_a_pod -n tenant-a -- /bin/bash -c "python3 /tmp/app.py"

In [ ]:
%%bash
export tenant_b_pod=$(kubectl get pods -n tenant-b -o jsonpath='{.items[0].metadata.name}')
kubectl exec -it pod/$tenant_b_pod -n tenant-b -- /bin/bash -c "python3 /tmp/app.py"